# Bias de colisión

> Bias: a systematic distortion of a statistical result due to a factor not allowed for in its derivation.

Pareciera que los estudios científicos más sonados son los menos confiables. Mientras más aburrido el tema, más rigurosos los resultados. ¿Cómo es que esta asociación negativa existe?

Se pueden pensar en muchas posibilidades, sin embargo, lo único necesario para que se de esta correlación negativa es que los pares revisores tomen en consideración tanto la *pantalla* que tenga el área de estudio, como la confiabilidad del estudio. En este escenario, el acto de selección será suficiente para que los estudios de mayor interés sean los menos confiables.


> **Objetivos:**
> - Entender el problema de bias de colisión.


> **Referencias:**
> 
> - Statistical Rethinking: A Bayesian Course with Examples in R and Stan (2nd edition) - Richard McElreath.

## 1. Introducción

Comprobémoslo mediante una simulación. Supongamos que un panel de revisión recibe 200 propuestas de investigación. En dichas propuestas, no existe correlación alguna entre la confiabilidad y el interés público. En este panel se le da importancia a la confiabilidad y al interés público de manera equitativa, y luego se ordenan las propuestas por sus puntajes combinados para seleccionar el 10% más alto para financiarlos.

In [ ]:
# Importar scipy.stats.norm

# Importar numpy

# Importar pyplot


In [ ]:
# Número de propuestas

# Proporción

# Simulación de interés público y confiabilidad

# Score combinado

# Selección del top 10%


In [ ]:
# Visualización


In [ ]:
# Correlación


Este mismo fenómeno de selección puede suceder en un modelo estadístico. Cuando pasa, puede distorcionar bastante nuestras inferencias.

## 2. Bias de colisión

Un DAG que modela la situación anterior es:

In [ ]:
# Importar daft
import daft
# Importar causalgraphicalmodels
from causalgraphicalmodels import CausalGraphicalModel

In [ ]:
# Función para dibujar modelos gráficos
def draw_dag(dag, coordinates):
    pgm = daft.PGM()
    for node in dag.dag.nodes:
        pgm.add_node(node, node, *coordinates[node])
    for edge in dag.dag.edges:
        pgm.add_edge(*edge)
    pgm.render()
    plt.gca().invert_yaxis()

In [ ]:
sci_dag = CausalGraphicalModel(
    nodes=["T", "S", "N"],
    edges=[("T", "S"), ("N", "S")]
)
coordinates = {"T": (0, 0), "S": (2, 0), "N": (4, 0)}

draw_dag(sci_dag, coordinates)

La confiabilidad $T$ y el interés público $N$ no están asociados, pero ambos influencian la selección $S$. Se llama **colisión** porque las flechas apuntan hacia $S$.

El concepto es: cuando se condiciona sobre el colisionador, se crea una asociación estadística (no necesariamente causal) entre las causas. Como vimos, una vez tenemos al top 10% de las propuestas, conocer su confiabilidad también nos da información de su interés público.

Esto significa que tenemos que prestar atención a los procesos que seleccionan explícita o implícitamente las muestras de nuestras observaciones, ya que pueden distorcionar las asociaciones entre las variables. También, este mismo fenómeno generará asociaciones falsas en un modelo estadístico, si incluimos al colisionador como predictor.

**¿La felicidad cambia con la edad?**

Consideremos ahora la pregunta de como envejecer influencia nuestra felicidad. Si tenemos una encuesta donde las personas declaran qué tan felices son, ¿Estará su edad asociada con la felicidad? Si sí, ¿es esta asociación causal?

Supongamos que la felicidad promedio es un rasgo de nacimiento, y no cambia con la edad. Sin embargo, la felicidad influencia los eventos de la vida misma. Por ejemplo, el matrimonio. La gente más feliz es más propensa a casarse. Otra variable que influencia el matrimonio es la edad. Mientras más años vives, más probable es que te cases. De modo que tenemos el siguiente modelo:

In [ ]:
happiness_dag = CausalGraphicalModel(
    nodes=["H", "M", "A"],
    edges=[("H", "M"), ("A", "M")]
)
coordinates = {"H": (0, 0), "M": (2, 0), "A": (4, 0)}

draw_dag(happiness_dag, coordinates)

La felicidad $H$ y la edad $A$ causan el matrimonio $M$. Por tanto, el matrimonio es un colisionador. Incluso, si no hay una asociación entre felicidad y la edad, si condicionamos en el matrimonio (lo incluimos como predictor en la regresión), induciremos una asociación estadística entre la edad y la felicidad, distorsionando la interpretación de que la felicidad cambia con la edad, cuando en realidad es constante.

Veámoslo con una simulación, nuevamente. Las simulaciones son bastante útiles en este tipo de ejemplos, dado que son los únicos casos donde conocemos el modelo causal verdadero. Si un procedimiento no puede revelar la verdad en un ejemplo simulado, no le deberíamos confiar un caso real. La simulación consiste en lo siguiente:

1. Cada año, nacen 20 personas con valores de felicidad uniformemente distribuidos.
2. Cada año, cada persona envejece un año. La felicidad no cambia.
3. A los 18 años de edad, las personas se pueden casar. La probabilidad de casarse cada año son proporcionales a la felicidad del individuo.
4. Una vez casado, el individuo permanece casado.
5. Después de los 65 años, el individuo deja la muestra.

La simulación está en `helper.py`, en la función `sim_happiness`.

In [ ]:
# Importar helper.sim_happiness

# Importar arviz


In [ ]:
# Simular datos


In [ ]:
# Summary
happiness_data["married"] = happiness_data["married"].astype(int)
az.summary(happiness_data.to_dict(orient="list"), kind="stats", round_to=2)

In [ ]:
happiness_data.shape

Estos son los datos de 1300 personas de todas las edades entre 0 a 65 años. Las variables siguen el modelo descrito en el DAG de arriba.

Para visualizar los datos con estas tres variables haremos un gráfico un tanto ingenioso, con las variables de edad y felicidad en los ejes $x$ y $y$, respectivamente, y pintando los puntos de color cuando el individuo está casado:

In [ ]:
happiness_data["married"] = happiness_data["married"].astype(bool)

In [ ]:
# Gráfica
fig, ax = plt.subplots(figsize=[10, 3.4])

colors = np.array(["w"] * happiness_data.shape[0])
colors[happiness_data["married"]] = "b"
ax.scatter(happiness_data["age"], happiness_data["happiness"], edgecolor="k", color=colors)

ax.scatter([], [], edgecolor="k", color="w", label="unmarried")
ax.scatter([], [], edgecolor="k", color="b", label="married")
ax.legend(loc="upper left", framealpha=1, frameon=True)

ax.set_xlabel("age")
ax.set_ylabel("happiness");

Supongamos que nos preguntamos si la edad se relaciona con la felicidad. No conocemos el modelo causal verdadero.

Consideramos una regresión lineal múltiple, para inferir la influencia de la edad en la felicidad, controlado para el estado marital:

$$
\mu_i = \alpha_{M[i]} + \beta_A A_i
$$

donde $M[i]$ es una variable índice que indica el estado marital de $i$.

Pensemos en la previa para $\beta_A$, no sin antes escalar la edad en el intervalo [0, 1]:

In [ ]:
adult_happiness_data = happiness_data[happiness_data["age"] >= 18].copy()
adult_happiness_data["age_unit"] = (adult_happiness_data["age"] - 18) / (65 - 18)

Tenemos entonces que $A$ varía de 0 (18) a 1 (65). La escala de la felicidad va de -2 a +2, de forma que la relación más fuerte que podríamos encontrar sería $\frac{2 - (-2)}{1} = 4$.

Una previa que cubriría estos casos extremos sería:

$$
\beta_A \sim \text{Normal}(0, 2).
$$

Ahora, cada $\alpha$ es el valor de $\mu_i$ cuando $A_i=0$. En ese caso, debería $\alpha$ poder tomar todos los posibles valores de felicidad (de -2 a +2):

$$
\alpha_{M[i]} \sim \text{Normal}(0, 1).
$$

In [ ]:
# Importar pymc


In [ ]:
adult_happiness_data["married"] = adult_happiness_data["married"].astype(int)
# Modelo


In [ ]:
# Summary


Observamos que el modelo nos muestra una gran seguridad de una asociación negativa entre la edad y la felicidad. Ahora, si descartamos el estado marital:

In [ ]:
# Modelo


In [ ]:
# Summary


Como contraste, vemos que en realidad no hay asociación entre edad y felicidad.

Esta patrón nos muestra exatamente lo que esperamos al condicionar (incluir como predictor) a un colisionador (estatus marital). Como vemos, al condicionar en un colisionador se induce una asociación falsa entre las dos causas.